# Import CSVs into OWID Grapher

The following notebook contains code for importing the UN SDG database into Grapher. It consumes the CSVs generated by a separate notebook, and the PDFs associated with each indicator.

### Notes

  - The length of the `name` column in the `datasets` table had to be changed to `512`: we have a `UNIQUE` constraint on it and the `namespace` column, and I was getting duplicates: `alter table datasets change name name varchar(512);`
  - We decided that this version of SDG needs to be imported into a new namespace (`un_sdg_new`), as it was not possible to update the values of the existing version of SDG that is present in Grapher.

In [ ]:
import mysql.connector
import pandas as pd
from datetime import datetime
import json

from import_metadata import extract_description

db = mysql.connector.connect(passwd="",db="owid", user="root", host="127.0.0.1")

pd.set_option('display.max_colwidth', -1)

# ID of admin user (Max)
USER_ID = 2

source_description = {
    'dataPublishedBy': "United Nations Statistics Division",
    'dataPublisherSource': None,
    'link': "https://unstats.un.org/sdgs/indicators/database/",
    'retrievedDate': datetime.now().strftime("%d-%B-%y"),
    'additionalInfo': None
}

QUERY_INSERT_DATASET = """
INSERT INTO datasets (name, description, createdAt, updatedAt, namespace, createdByUserId, metadataEditedAt, metadataEditedByUserId, dataEditedAt, dataEditedByUserId)
VALUES (%s, 'This is a dataset imported by the automated fetcher', NOW(), NOW(), 'un_sdg_new', %s, NOW(), %s, NOW(), %s)
"""

QUERY_INSERT_VARIABLE = """
INSERT INTO variables (name, unit, description, createdAt, updatedAt, coverage, 
                       timespan, datasetId, sourceId, display, columnOrder)
VALUES (%s, %s, NULL, NOW(), NOW(), '', '', %s, %s, '{}', 0)
"""

QUERY_INSERT_SOURCE = """
INSERT INTO sources (name, description, createdAt, updatedAt, datasetId)
VALUES (%s, %s, NOW(), NOW(), %s)
"""

QUERY_INSERT_DATAPOINTS = """
INSERT INTO data_values (value, year, entityId, variableId)
VALUES (%s, %s, %s, %s)
"""


## Read the results of the pre-processing script

In [ ]:
variables = pd.read_csv('./exported_data/variables.csv')
datasets = pd.read_csv('./exported_data/datasets.csv')

In [ ]:
for i, dataset in datasets.iterrows():
    cursor = db.cursor()
    additional_info = ''
    try:
        additional_info = extract_description('metadata/Metadata-%s.pdf' % '-'.join([part.rjust(2, '0') for part in dataset['Indicator'].split('.')]))
    except:
        print("Couldn't find metadata for indicator %s" % dataset['Indicator'])
        
    try:
        # insert row in datasets table
        cursor.execute(QUERY_INSERT_DATASET, ("%s %s" % (dataset.Indicator, dataset.SeriesDescription), USER_ID, USER_ID, USER_ID))
        new_dataset_id = cursor.lastrowid
        print("Inserted dataset: %s" % dataset.SeriesDescription)

        source_description['additionalInfo'] = additional_info

        # insert row in sources table
        cursor.execute(QUERY_INSERT_SOURCE, 
                      (dataset.SeriesDescription[:250], json.dumps(source_description), new_dataset_id))
        new_source_id = cursor.lastrowid

        print("Inserted source: %d" % new_source_id)

        # Insert variables associated with this dataset
        for j, variable in variables[(variables.Indicator == dataset.Indicator) & (variables.SeriesCode == dataset.SeriesCode)].iterrows():
            # insert row in variables table
            cursor.execute(QUERY_INSERT_VARIABLE, 
                          (variable.VariableDescription, variable.Units, new_dataset_id, new_source_id))
            new_variable_id = cursor.lastrowid
            print("Inserted variable: %s" % variable.VariableDescription)

            # read datapoints
            print("Reading datapoints: %04d_datapoints.csv" % variable.variable_idx)
            data_values = pd.read_csv('./exported_data/%04d_datapoints.csv' % variable.variable_idx)
            
            # keep columns that matter
            data_values = data_values[[
                'Value',
                'TimePeriod',
                'owid_entity_id',
            ]]
            
            # there are a few datasets that contain duplicates for the same country and year
            # also, drop the ocassional NaN in any of the columns
            # Clean them up
            data_values = data_values.drop_duplicates(subset=['TimePeriod', 'owid_entity_id']).dropna()

            values = [(value.Value, value.TimePeriod, value.owid_entity_id, new_variable_id)
                      for _, value in data_values.iterrows()]

            print("Inserting values...")
            cursor.executemany(QUERY_INSERT_DATAPOINTS, values)
            print("Inserted %d values for variable" % len(values))

        db.commit()
        cursor.close()
        print("------")
        
    except Exception as e:
        db.rollback()
        raise e


### Useful SQL for deleting all `data_values`, `variables`, `sources`, and `datasets` belonging to a `namespace`

```
DELETE data_values
FROM   data_values
       INNER JOIN variables
               ON variables.id = data_values.variableid
       INNER JOIN sources
               ON sources.id = variables.sourceid
       INNER JOIN datasets
               ON datasets.id = sources.datasetid
WHERE  datasets.namespace = 'un_sdg_new';

DELETE variables
FROM   variables
       INNER JOIN sources
               ON sources.id = variables.sourceid
       INNER JOIN datasets
               ON datasets.id = sources.datasetid
WHERE  datasets.namespace = 'un_sdg_new';

DELETE sources
FROM   sources
       INNER JOIN datasets
               ON datasets.id = sources.datasetid
WHERE  datasets.namespace = 'un_sdg_new';

DELETE FROM datasets
WHERE  namespace = 'un_sdg_new';  
```